In [21]:
import pandas as pd
import numpy as np
import h5py
from gtda.images import RadialFiltration
from gtda.homology import CubicalPersistence

In [22]:
subject = "m292"

In [23]:
dataframes = {}

label_list = [0,1,2,3,4] 

n_folds = 5

for label in label_list:
    filename = "Data/"+str(subject)+"/run0"+str(label)+"/Brain_Imaging_Data.h5"
    file = h5py.File(filename,'r')
    dataframes[label] = file['Data']

# Compute persistence diagram for each time step separately

In [26]:
def compute_persistence_diagrams(dataframes, label_list):

    radial_filtration = RadialFiltration(center=np.array([150, 150]))

    persistence_diagrams = {}
    
    for label in label_list:
        persistence_diagrams[label] = []
        for image in dataframes[label]:#[:100]: # Cut here if you do not want to use the complete data
            filtration = radial_filtration.fit_transform([image])
            cubical_persistence = CubicalPersistence(n_jobs=-1)
            diagram = cubical_persistence.fit_transform(filtration)
            persistence_diagrams["Label_"+str(label)].append(diagram[0])


    return persistence_diagrams


persistence_diagrams = compute_persistence_diagrams(dataframes, label_list)

In [27]:
np.save("Embeddings_and_Persistence_Diagrams/"+str(subject)+"/Persistence_Diagrams.npy", np.array(persistence_diagrams, dtype=object), allow_pickle=True)